In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math



/Users/R1/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
#define FixedWidthVariables class which reads & maps Stata dictionary files into pandas

class FixedWidthVariables(object):

    def __init__(self, variables, index_base = 0):
        self.variables = variables
        self.colspecs = variables[['start','end']] - index_base
        self.colspecs = self.colspecs.astype(np.int).values.tolist()
        self.names = variables['name']

    def ReadFixedWidth(self, filename, **options):
        df = pd.read_fwf(filename, colspecs = self.colspecs, names = self.names, **options)

        return df



In [4]:
#define StatData to parse through Stata dct file and extract variable names 

def StatData(dct_file, **options):
    types = dict(byte=int, int=int, long=int, float=int, double=float)

    var_info = []
    for line in open(dct_file, **options):
        row = re.search(r'_column\(([^)]*)\)', line)
        if row:
            start = int(row.group(1))
            t = line.split()
            vtype, name, fstring = t[1:4]
            name = name.lower()
            if vtype.startswith('str'):
                vtype = str
            else:
                vtype = types[vtype]
            long_desc = ' '.join(t[4:]).strip('"')
            var_info.append((start, vtype, name, fstring, long_desc))

    columns = ['start', 'type','name','fstring','desc']
    variables = pd.DataFrame(var_info, columns = columns)

    variables['end'] = variables.start.shift(-1)
    variables.loc[len(variables)-1, 'end'] = 0
    dct = FixedWidthVariables(variables, index_base = 1)
    return dct


In [5]:
# CleanData removes noise from the datafile and standardizes some numbers

def CleanData(df):
    df.agepreg /= 100.0
    df.loc[df.birthwgt_lb > 20, 'birthwgt_lb'] = np.nan
    replacement = [97, 98, 99]
    df.birthwgt_lb.replace(replacement, np.nan, inplace=True)
    df.birthwgt_oz.replace(replacement, np.nan, inplace=True)
    df.hpagelb.replace(replacement, np.nan, inplace=True)
    df.babysex.replace([7,9], np.nan, inplace=True)
    df.nbrnaliv.replace([9], np.nan, inplace=True)
    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0
    df.cmintvw = np.nan

In [6]:
# cohen's d statistic  

def cohens_d(array_1, array_2):
    len1 = len(array_1)
    len2 = len(array_2)
    mean1 = array_1[~np.isnan(array_1)].mean()
    var1 = array_1[~np.isnan(array_1)].var()
    std1 = array_1[~np.isnan(array_1)].std()
    mean2 = array_2[~np.isnan(array_2)].mean()
    var2 = array_2[~np.isnan(array_2)].var()
    std2 = array_2[~np.isnan(array_2)].std()
    net_var = (len1*var1+len2*var2)/(len1+len2)
    mean_diff = mean1 - mean2
    print ' the mean difference is ', mean_diff
    d = mean_diff/math.sqrt(net_var)
    return d

In [7]:
labels = '2002FemPreg.txt'
data = '2002FemPreg.dat.gz'

dct = StatData(labels)


raw_data = dct.ReadFixedWidth(data, compression = 'gzip')
CleanData(raw_data)
live = raw_data[raw_data.outcome == 1]
first = live[live.birthord == 1]
rest = live[live.birthord != 1]

weight_diff = cohens_d(first.totalwgt_lb.as_matrix(), rest.totalwgt_lb.as_matrix())
gest_diff = cohens_d(first.prglngth.as_matrix(), rest.prglngth.as_matrix())

print "cohen's D for birth weight between first children and others is ", weight_diff
print "cohen's D for pregnancy length between first children and others is ", gest_diff

 the mean difference is  -0.124761184535
 the mean difference is  0.0780372667775
cohen's D for birth weight between first children and others is  -0.0886827459471
cohen's D for pregnancy length between first children and others is  0.0288822092882


In [11]:
plt.hist(live.prglngth)
plt.title('Pregnancy Length for Live Births')
plt.xlabel('Weeks')
plt.ylabel('Frequency')
plt.show()